# Sequential Workflow with Shared Conversation Context

## Overview

This notebook demonstrates **sequential orchestration** using the `SequentialBuilder` API. Unlike concurrent workflows where agents work in parallel, sequential workflows pass a shared conversation context through each participant in order. Each agent appends its response to the growing conversation.

### Key Concepts:

1. **SequentialBuilder**: High-level API for building linear workflows
2. **Shared Context**: `list[ChatMessage]` flows through all participants
3. **Incremental Building**: Each agent sees previous messages
4. **Internal Adapters**: Small nodes for input normalization and completion
5. **Final Output**: Complete conversation history

### Architecture:

```
User Prompt
    ↓
[input-conversation adapter]
    ↓
Writer Agent
    ↓
[to-conversation:writer adapter]
    ↓
Reviewer Agent
    ↓
[to-conversation:reviewer adapter]
    ↓
[complete adapter]
    ↓
Final Conversation (list[ChatMessage])
```

### Internal Adapters Explained:

The sequential orchestration includes small adapter nodes that may appear in the event stream:
- **`input-conversation`**: Normalizes input into conversation format
- **`to-conversation:<participant>`**: Converts agent response to ChatMessage
- **`complete`**: Signals completion and yields final output

You can safely ignore these when focusing on agent progress.

### When to Use Sequential Orchestration:

- **Iterative refinement**: Each step builds on previous work
- **Writer-reviewer patterns**: Draft → Review → Edit flows
- **Progressive enhancement**: Incrementally add detail or analysis
- **Context-dependent tasks**: Later steps need earlier results

## Prerequisites

- Azure OpenAI configured with environment variables
- Azure CLI authentication: Run `az login` before executing
- Agent Framework installed: `pip install agent-framework`

## Setup and Imports

In [ ]:
import asyncio
from typing import cast

from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Create Sequential Agents

### Writer Agent
- Produces concise, punchy marketing copy
- First in the sequence
- Creates initial content

### Reviewer Agent
- Provides thoughtful feedback
- Second in the sequence
- Sees both user prompt and writer's response

In [ ]:
async def run_sequential_workflow() -> None:
    # Create Azure OpenAI chat client
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    writer = chat_client.create_agent(
        instructions=("You are a concise copywriter. Provide a single, punchy marketing sentence based on the prompt."),
        name="writer",
    )

    reviewer = chat_client.create_agent(
        instructions=("You are a thoughtful reviewer. Give brief feedback on the previous assistant message."),
        name="reviewer",
    )

## Build Sequential Workflow

### Key Points:

- **Order matters**: Agents execute in the order specified
- **Shared conversation**: Each agent receives the full conversation history
- **Append-only**: Each agent adds its response to the conversation

In [ ]:
    # Build sequential workflow: writer -> reviewer
    workflow = SequentialBuilder().participants([writer, reviewer]).build()

## Execute Workflow with Streaming

We'll use streaming to observe the workflow execution and collect the final conversation.

### Streaming Benefits:
- See incremental progress
- Capture intermediate events
- Monitor workflow state transitions

In [ ]:
    # Run and collect outputs using streaming
    outputs: list[list[ChatMessage]] = []
    async for event in workflow.run_stream("Write a tagline for a budget-friendly eBike."):
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(list[ChatMessage], event.data))

    if outputs:
        print("===== Final Conversation =====")
        for i, msg in enumerate(outputs[-1], start=1):
            name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
            print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")

## Run the Complete Workflow

In [ ]:
await run_sequential_workflow()

## Expected Output

```
===== Final Conversation =====
------------------------------------------------------------
01 [user]
Write a tagline for a budget-friendly eBike.
------------------------------------------------------------
02 [writer]
Ride farther, spend less—your affordable eBike adventure starts here.
------------------------------------------------------------
03 [reviewer]
This tagline clearly communicates affordability and the benefit of extended travel, making it
appealing to budget-conscious consumers. It has a friendly and motivating tone, though it could
be slightly shorter for more punch. Overall, a strong and effective suggestion!
```

## Key Takeaways

### 1. Sequential Builder Simplicity
- One line to create workflow: `SequentialBuilder().participants([...]).build()`
- Order of participants defines execution sequence
- Automatic conversation management

### 2. Shared Conversation Context
- Each agent receives complete `list[ChatMessage]` history
- Agents can reference previous responses
- Context grows with each participant

### 3. Internal Adapter Nodes
- Framework adds helper executors for:
  - Input normalization (`input-conversation`)
  - Response conversion (`to-conversation:<name>`)
  - Completion signaling (`complete`)
- These appear in event streams but can be ignored
- Similar to concurrent's dispatcher/aggregator

### 4. Message Authorship
- `author_name` property identifies which agent responded
- User messages have no `author_name` (or generic "user")
- Assistant messages tagged with agent name

### 5. Output Structure
- Final output is `list[ChatMessage]`
- Contains all messages in chronological order:
  1. User's original prompt
  2. Writer's response
  3. Reviewer's feedback

### 6. Workflow Completion
- Completes when all participants finish
- No pending messages to process
- Final conversation available via `WorkflowOutputEvent`

### 7. Use Cases
- **Draft-Review-Edit**: Writer → Reviewer → Editor pipelines
- **Progressive Summarization**: Extract → Summarize → Refine
- **Multi-stage Analysis**: Research → Analysis → Recommendations
- **Iterative Content**: Generate → Critique → Improve

### 8. Comparison with Concurrent

| Feature | Sequential | Concurrent |
|---------|-----------|------------|
| **Execution** | One at a time, in order | All simultaneously |
| **Context** | Shared, growing conversation | Independent, parallel |
| **Dependencies** | Later steps see earlier results | No inter-agent dependencies |
| **Speed** | Slower (serial) | Faster (parallel) |
| **Use Case** | Refinement, building context | Independent analysis |

### 9. Customization Options
- Mix agents and custom executors (see sequential_custom_executors.py)
- Add custom processing between agents
- Combine with concurrent stages for hybrid workflows

### 10. Production Considerations
- Monitor conversation length (token accumulation)
- Consider truncation for very long sequences
- Add error handling for individual agent failures
- Log each agent's contribution separately
- Set timeouts for slow agents